#1

In [1]:
!pip install razdel

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import re
from razdel import tokenize

In [3]:
from typing import List

In [5]:
data = pd.read_csv('labeled.csv')

In [6]:
def custom_tokenizer(s: str) -> list:
    return [token.text for token in tokenize(s)]

In [ ]:
vectorizer = TfidfVectorizer(min_df=10, max_features=100, max_df=0.3)
vectorizer_custom = TfidfVectorizer(min_df=10, max_features=100, max_df=0.3, tokenizer=custom_tokenizer)

In [9]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

X_custom = vectorizer_custom.fit_transform(train.comment)
X_test_custom = vectorizer_custom.transform(test.comment)

In [12]:
y = train.toxic.values
y_test = test.toxic.values

In [ ]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X, y)
clf_custom = LogisticRegression(C=0.1, class_weight='balanced')
clf_custom.fit(X_custom, y)

In [ ]:
preds = clf.predict(X_test)
preds_custom = clf_custom.predict(X_test_custom)

In [ ]:
print(classification_report(y_test, preds, zero_division=0))

In [ ]:
print(classification_report(y_test, preds_custom, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.82      0.69      0.75       952
         1.0       0.54      0.70      0.61       490

    accuracy                           0.70      1442
   macro avg       0.68      0.70      0.68      1442
weighted avg       0.72      0.70      0.70      1442



дефолтный лучше везде

#2

In [7]:
count_vectorizer = CountVectorizer(
    min_df=10,
    max_features=100,
    max_df=0.5,
    ngram_range=(1, 3),
    tokenizer=custom_tokenizer
)

tfidf_vectorizer = TfidfVectorizer(
    min_df=10,
    max_features=100,
    max_df=0.5,
    ngram_range=(1, 3),
    tokenizer=custom_tokenizer
)

In [46]:
train2, test2, train2_, test2_ = train_test_split(
    data['comment'],
    data['toxic'],
    test_size=0.2,
    random_state=42
)

In [47]:
X_count = count_vectorizer.fit_transform(train2)
X_test_count = count_vectorizer.transform(test2)
X_tfidf = tfidf_vectorizer.fit_transform(train2)
X_test_tfidf = tfidf_vectorizer.transform(test2)

In [48]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X_count, train2_)
rf = RandomForestClassifier(n_estimators=100, max_depth=20)
rf.fit(X_tfidf, train2_)

RandomForestClassifier(max_depth=20)

In [59]:
preds_clf = clf.predict(count_vectorizer.transform(test2))
preds_rf = rf.predict(tfidf_vectorizer.transform(test2))


In [57]:
print(classification_report(test2_, preds_clf, zero_division=0))
print(classification_report(test2_, preds_rf, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.85      0.64      0.73      1944
         1.0       0.51      0.76      0.61       939

    accuracy                           0.68      2883
   macro avg       0.68      0.70      0.67      2883
weighted avg       0.74      0.68      0.69      2883

              precision    recall  f1-score   support

         0.0       0.76      0.90      0.82      1944
         1.0       0.66      0.39      0.49       939

    accuracy                           0.74      2883
   macro avg       0.71      0.65      0.66      2883
weighted avg       0.73      0.74      0.72      2883



In [63]:
probas_clf = clf.predict_proba(count_vectorizer.transform(test2))[:, 1]
probas_rf = rf.predict_proba(tfidf_vectorizer.transform(test2))[:, 1]

In [17]:
import numpy as np

In [68]:
top10_clf = np.argsort(probas_clf)[::-1][:10]
top10_rf = np.argsort(probas_rf)[::-1][:10]

In [71]:
from pprint import pprint

In [74]:
clf_top = [test2.iloc[i] for i in top10_clf]
for i in clf_top:
    print(i[:100])

Предлагаю вашему вниманию поэтические страницы маестра. До тридцати сиянье чуда В тебе, быть может, 
Нет, пожалуй, отвечу развернутее. Представь, что тебя ограбили. Ты идешь в частную полицию, выкладыв
Раз уж вакханалия продолжается, давайте в этом треде продолжать собирать ворох банов. Копирую из про
И? У него свобода передвижения, а у водителей требование 1.5, водить ТС так, что бы никому не вредит
ой блять, как же вы заебали один еблан впишется в тему и прилично устроится и всё!1 - ОН, БЛЯТЬ, ВС 
По мексикански Флаг: Ублюдок, мать твою, а ну иди сюда говно собачье, решил меня поднять? Ты, засран
Ты серьёзно думаешь, что из-за того, что ты смотришь Панчина ты не относишься к 95 населения? Ну смо
лучше скажи мне нахуй кто-то смотрит эти дёргания инвалидки по несколько часов ты смотришь, долбоеб,
Модератор - уровень Якен Хгар - Нихуя себе, всего три убийства! Дяденька, ну ты, бля, и жлоб - Следи
Т.е. приветствовали нарушение установленных показателей по производству, добыче и т.п.? отк

In [75]:
rf_top = [test2.iloc[i] for i in top10_rf]
for i in rf_top:
    print(i[:100])

Кто мы то? Ты обосрался, иди мойся под струю.

быдлокоммидаун Промытая малолетка, ты?

ебать ты конченый, нахуя ты это постишь?

Ты просто куколд ебаный

Ты просто шизик. Скройся.

Доиграешься ты со своей русофобией.

Возможно, ты красивая

Теперь ты модератор бакланов.

ты чо дурачочек чтоле

Нет, ты именно другое сказал. Извинись перед всеми



моделька rf гораздо лучше справилась с нахождением токсичных коментов. у clf оч длинные тексты бтв, мб там не в лексике токсичность, а в посыле.

In [76]:
for i in rf_top:
    if i in clf_top:
        print(i)

ничего. ура (i hate my life).
on a serious note, просто оч разные подходы у моделек